In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.max_colwidth', 0)

import chart_studio
chart_studio.tools.set_credentials_file(username='YourUserName', api_key='YourAPIKey')
import chart_studio.plotly as py

In [ ]:
df_master_company_sel = pd.read_pickle('Data_2021_updated.pkl')

In [ ]:
df_master_company_sel.head(2)

In [ ]:
df_master_company_sel['year_of_registration'] = df_master_company_sel['date_of_registration'].apply(lambda x:x.split('-')[0])
df_master_company_sel['year_of_registration'] = df_master_company_sel['year_of_registration'].astype(int)

df_master_company_sel = df_master_company_sel[df_master_company_sel['year_of_registration']>=2000]
df_master_company_sel = df_master_company_sel[df_master_company_sel['year_of_registration']<=2021]

df_master_company_sel = df_master_company_sel[['corporate_identification_number','company_name','company_status','company_class',
                      'company_category','date_of_registration','registered_state',
                       'principal_business_activity_as_per_cin','year_of_registration']]

df_master_company_sel = df_master_company_sel.drop_duplicates()


In [ ]:
len(df_master_company_sel)

In [ ]:
df_master_company_sel.head(2)

In [ ]:
df_master_company_sel.info()

In [ ]:
df_company_status = df_master_company_sel.groupby(['company_status'])['corporate_identification_number'].nunique().reset_index()
df_company_status = df_company_status.sort_values(by='corporate_identification_number')
df_company_status['title'] = df_company_status['company_status'].apply(lambda x:x.replace(' ','<br>'))

fig = px.bar(df_company_status,x='title',y='corporate_identification_number',text='corporate_identification_number')
fig.update_xaxes(title='Company Status')
fig.update_yaxes(title='Number of Companies')
fig.update_layout(xaxis = go.layout.XAxis(tickangle = 0))
fig.update_layout(plot_bgcolor='white')

fig.update_layout(
    font=dict(
        size=9
    )
)


In [ ]:
active_companies = df_master_company_sel[df_master_company_sel['company_status']=='Active']
prop = active_companies['corporate_identification_number'].nunique()/df_master_company_sel['corporate_identification_number'].nunique()
round(prop*100)


In [ ]:
df_active_companies = df_master_company_sel[df_master_company_sel['company_status']=='Active']

In [ ]:
df_active_companies['year_of_registration'] = df_active_companies['date_of_registration'].apply(lambda x:x.split('-')[0])
df_active_companies['year_of_registration'] = df_active_companies['year_of_registration'].astype(int)

df_yearly_registrations = df_active_companies.groupby('year_of_registration')['corporate_identification_number'].nunique().reset_index()

fig = px.bar(df_yearly_registrations,x='year_of_registration',
             y='corporate_identification_number')
fig.update_xaxes(title='Year of registration')
fig.update_yaxes(title='Number of Companies')
fig.update_layout(plot_bgcolor='white')
fig.update_traces(marker_color='green')


In [ ]:
df_state_registrations = df_active_companies.groupby(['registered_state'])['corporate_identification_number'].nunique().reset_index()
df_state_registrations = df_state_registrations.sort_values(by='corporate_identification_number')

fig = px.bar(df_state_registrations,x='registered_state',
             y='corporate_identification_number', text='corporate_identification_number')
fig.update_xaxes(title='Registered State')
fig.update_yaxes(title='Number of Companies')
fig.update_layout(xaxis = go.layout.XAxis(tickangle = 90))
fig.update_layout(plot_bgcolor='white')
fig.update_traces(marker_color='purple')

fig.update_layout(
    font=dict(
        size=10
    )
)

In [ ]:
df_active_companies['principal_business_activity_as_per_cin'].nunique()

In [ ]:
df_active_companies['principal_business_activity_as_per_cin'].unique()

In [ ]:
business_activity_active = df_active_companies.groupby(['principal_business_activity_as_per_cin'])['corporate_identification_number'].nunique().reset_index()
business_activity_active = business_activity_active.sort_values(by='corporate_identification_number')
business_activity_active['%'] = business_activity_active['corporate_identification_number']*100/df_active_companies['corporate_identification_number'].nunique()
business_activity_active = business_activity_active.round(3)
business_activity_active.columns = ['Business Activity','# of Companies','% of total active companies']

In [ ]:
business_activity_active.sort_values(by='% of total active companies').tail(5)

In [ ]:
not_active_companies = df_master_company_sel[df_master_company_sel['company_status']!='Active']
df1 = not_active_companies.groupby(['principal_business_activity_as_per_cin'])['corporate_identification_number'].nunique().reset_index()
df2 = df_master_company_sel.groupby(['principal_business_activity_as_per_cin'])['corporate_identification_number'].nunique().reset_index()

df1.columns = ['business_activity','non_active_company_count']
df2.columns = ['business_activity','total_company_count']

df_activity = pd.merge(df1,df2,on='business_activity')
df_activity['%'] = df_activity['non_active_company_count']*100/df_activity['total_company_count']


In [ ]:
df_activity.sort_values(by='%',ascending=False).round().head(5)


In [ ]:
df_activity.sort_values(by='%',ascending=False).round().tail(5)

In [ ]:
df_active_companies.head(2)

In [ ]:
df_2000 = df_active_companies[df_active_companies['year_of_registration']==2000]
df_2020 = df_active_companies[df_active_companies['year_of_registration']==2020]

df_2000 = df_2000.groupby('registered_state')['corporate_identification_number'].nunique().reset_index()
df_2020 = df_2020.groupby('registered_state')['corporate_identification_number'].nunique().reset_index()
df_total = df_active_companies.groupby('registered_state')['corporate_identification_number'].nunique().reset_index()

df_2000.columns = ['state','2000_companies']
df_2020.columns = ['state','2020_companies']
df_total.columns = ['state','total_companies']

df_combined = pd.merge(df_2000,df_2020,on='state')
df_combined = pd.merge(df_combined,df_total,on='state')

df_combined['% change'] = (df_combined['2020_companies'] - df_combined['2000_companies'])/df_combined['2000_companies']
df_combined['% change'] = (df_combined['% change']*100).round(2)

fig = px.scatter(df_combined,x='2000_companies',y='2020_companies',
                 size='total_companies',hover_name='state')

fig.update_xaxes(title='Number of companies registered in 2000')
fig.update_yaxes(title='Number of companies registered in 2020')

fig.update_layout(plot_bgcolor='white')
fig.update_traces(marker_color='purple')

fig.update_layout(
    font=dict(
        size=12
    )
)


py.plot(fig, filename = 'PercentageChange', auto_open=True)

In [ ]:
df_combined.sort_values(by='% change').head(5)

In [ ]:
df_combined.sort_values(by='% change').tail(5)